In [3]:
###  Gradient Descent python implementation
import numpy as np

def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean( y_true*np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))

In [4]:
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))
    
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [5]:
### to find w1 , w2 , bias 
def gradient_descent(age , affordability , y_true,epochs):
    w1 = w2 = 1
    bias = 0
    #learning rate
    rate = 0.5
    n = len(age)

    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true,y_predicted)
        
        w1_derivative = (1/n) * np.dot(np.transpose(age),(y_predicted-y_true))
        w2_derivative = (1/n) * np.dot(np.transpose(affordability),(y_predicted-y_true))
        bias_derivative = np.mean(y_predicted-y_true)

        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print(f'Epoch:{i}, w1:{w1} , w2:{w2}, bias:{bias}, loss{loss}' )